In [1]:
import os
import random
import numpy as np
import glob
import unidecode
import string
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from tensorboardX import SummaryWriter

from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
model_name = 'bable-v3'
model_type = 'gru'
n_steps = 60000
print_steps = 1000
save_steps = 1000
hidden_size = 256
n_layers = 2
learning_rate = 1e-3
seq_len = 128
batch_size = 64
cuda = True
seed = 9999
data_filepath = 'D:\\Data\\northernlion\\output\\nl-isaac.txt'
save_filepath = 'D:\\Models\\Bable\\' + model_name
if not os.path.exists(save_filepath):
    os.makedirs(save_filepath)
log_filepath = 'D:\\Logs\\Bable\\' + model_name
if not os.path.exists(log_filepath):
    os.makedirs(log_filepath)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic=True

In [3]:
# TODO should process nl-isaac.txt to convert unicode that can't be printed to some equivalent
all_characters = string.printable
n_characters = len(all_characters)

def read_file(filepath):
    file = unidecode.unidecode(open(filepath).read())
    return file, len(file)

# Turning a string into a tensor
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

# Readable time elapsed
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return f'{m}m {s:.2f}s'

In [4]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model_type="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model_type = model_type.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model_type == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model_type == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, inputs, hidden):
        batch_size = inputs.size(0)
        encoded = self.encoder(inputs)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.out(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model_type == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        else:
            return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))


In [5]:
file, file_len = read_file(data_filepath)

In [6]:
print(f'Total file characters: {file_len}')

Total file characters: 71056008


In [7]:
def random_training_set(seq_len, batch_size):
    inp = torch.LongTensor(batch_size, seq_len)
    target = torch.LongTensor(batch_size, seq_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - seq_len)
        end_index = start_index + seq_len + 1
        seq = file[start_index:end_index]
        inp[bi] = char_tensor(seq[:-1])
        target[bi] = char_tensor(seq[1:])
    inp = Variable(inp)
    target = Variable(target)
    if cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    hidden = model.init_hidden(batch_size)
    if cuda:
        hidden = hidden.cuda()
    model.zero_grad()
    loss = 0

    for c in range(seq_len):
        output, hidden = model(inp[:, c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:, c])

    loss.backward()
    model_optimizer.step()
    return loss.data / seq_len

def save(model, step):
    model_filepath = os.path.join(save_filepath, f'{model_name}-{step}.pth.tar')
    #torch.save(model, model_filepath)
    torch.save({'state_dict': model.state_dict()}, model_filepath)
    print(f'Saved as {model_filepath}')
    
def load(model, step):
    model_filepath = os.path.join(save_filepath, f'{model_name}-{step}.pth.tar')
    checkpoint = torch.load(model_filepath)
    model.load_state_dict(checkpoint['state_dict'])


In [8]:
def generate(model, prime_str='A', predict_len=100, temperature=0.8, cuda=False):
    hidden = model.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    if cuda:
        hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[:,p], hidden)
        
    inp = prime_input[:, -1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted


In [9]:
# Initialize models and start training

model = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model_type=model_type,
    n_layers=n_layers,
)
model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

if cuda:
    model.cuda()

In [10]:
start = time.time()
writer = SummaryWriter(log_filepath)
try:
    print(f'Training for {n_steps} steps...')
    model.train()
    for step in tqdm(range(1, n_steps + 1)):
        loss = train(*random_training_set(seq_len, batch_size))
        writer.add_scalar('train/loss', loss, step)
        
        if step % print_steps == 0:
            print(f'[{time_since(start)} ({step} {(step / n_steps)*100:.2f}%) {loss:.2f}]')
            model.eval()
            sample_text = generate(model, 'hey', 2000, cuda=cuda)
            print(sample_text, '\n')
            writer.add_text('train/sample', sample_text, step)
            model.train()
            
        if step % save_steps == 0:
            save(model, step)
            
except KeyboardInterrupt:
    print('Stopping training early...')
finally:
    writer.close()
    print('Saving...')
    save(model, step)
    
model.eval()

Training for 60000 steps...


  2%|█▎                                                                          | 999/60000 [07:39<7:16:30,  2.25it/s]

 40.15s (1000 1.67%) 1.16]
hey never certainly being extra bandage consumable that  of the spirit or my card it's more bombs is advance is bitch that's all right you know for my happens in our bumb of a a run just indetertically and we can't be a try to do play goat much as would be like a completely op Sate interestand to make it worked out in here to be would be a strance of that room watching the etce that's okay that I just like to go basically much we did the weed at least now that considerstand it change but we can completely so but it considences that I was a little bit more of the large shop oh you want to fly will be a little HP to stay so I'm still got a foot lucky finstone and then you can't actually could be a with the bender now our for that I hate it time so I'm not surprises to the right now we have a guppy demon victory on my order the in the spending and we'll addest and or something for that boss rush ends that's okay I'm trying to be a bomb I have been so good or for

  3%|██▍                                                                        | 1999/60000 [15:12<7:36:29,  2.12it/s]

 12.64s (2000 3.33%) 1.11]
hey eaten lettere too shooting much being so let's let me this is pretty powered for money or in the same competitive altime all right let's go to find our red chest of the song on the moving on a form is just definitely we gotta be more good enemy and by oh I can't give us a little bit more for the mouth card to work I would be like you got a beautiful we're gonna say but it was a bad item and then we should get hit and then our surprisingly all right overly chest so I understand the episode I have to you know it leave and just in the extra luckie we're gonna be fine all a tears and that was not really in the future always in a bomb mathemation maybe sure we were feeling to look love to say the boss room because it's at one really good to do a something else we should's like hey I mean from I mean I have a second I ton a hard to it it's all cute for us okay that we could be able to me look that you know what I was like you know but we'd like the run is just 

...

100%|█████████████████████████████████████████████████████████████████████████▉| 59999/60000 [8:13:30<00:00,  1.98it/s]

 30.70s (60000 100.00%) 1.00]
hey've got a couple of brew kills I'm moving the early and all of our actually good benefit the occasional personal tight downtownet could be worse and it might be better myself in this run as quickly you know maybe the most important movie just stupid unless a stupid they did my business ready for you to like a piece of a bomb to have it on this run and we should take the bomb okay we got to stick with this and from there there I would love to get an arcade will use magic mushroom then provide the game shot speed you know the odds of good oh you know what let's not use that bomb or sight and if we just get a blood bank there um it is our damage causes us up here you know we got nothing wrong with the afterbirth for something that I would feel relatively lost the damage on this is a boss item so you know it doesn't matter now I'm clever in my early change my body yesterday in one of the most likelihood either just turned up in a part of me I'm not trying t

100%|██████████████████████████████████████████████████████████████████████████| 60000/60000 [8:13:33<00:00,  1.20s/it]


Saving...
Saved as D:\Models\Bable\bable-v3\bable-v3-60000.pth.tar


CharRNN(
  (encoder): Embedding(100, 256)
  (rnn): GRU(256, 256, num_layers=2)
  (out): Linear(in_features=256, out_features=100, bias=True)
)

In [11]:
generate(model, prime_str='hey', predict_len=2000, temperature=0.8, cuda=cuda)

"hey mean this is it that's very until the second secret room is standing right now we're at our boss trap rooms to still still probably we're not going to take damage but in the course of the game is unnecessary with that as well but that's a direct player I don't necessarily probably play the final course or something like that I actually accept that was check the start of the line please just for like one of those of you this are well that's just a song stop that consciously incredibly different show in many ways that's my word right now we go in that fight but I would be like there we go and more or less in the game now you just must be able to do that for the next floor key I asked for it was not tricking a little annoying waves the biggest items we can actually get the damage is a little bit of a we can do this we got an exactly what we're still using a spear at hard deals with the devil I'm definitely a little easier to double the quality of my percentage of Brimstone's like it 

In [12]:
generate(model, prime_str='devil', predict_len=2000, temperature=0.8, cuda=cuda)

"devil basically the twice and I should have been there to re-roll any red hearts but right here and this is where we stay going as well as well we're just going to afford to be not definitely gonna get there I think we could have taken that at for this I digress we're gonna do it if we picked up a shot speed upgrade which is a shoe pill here is the first place is looking actually so we have no benefit this way this is like the the future for now thanks for watching hope you enjoyed the episode if you did click the like button helps out a great deal of course subscribe if you want a while as well and Larry judgment do I'm still exist but it's been a white not as excited to do my ability to get you got oh fuck it starting to become guppy just because we will do things that doesn't end up being the cities at least I don't want to try something about it look it up a hundred percent or they go oh it was worth it the devil chance on the next floor anyway speed but you can just drop me basic